# A* 算法的应用与实践

这里是一些说明性的文本

In [1]:
import numpy as np
from heapq import heapify, heappop, heappush

np.random.seed(0)

In [5]:
def can_access(x, y):
    if x < 0 or x >=n or y < 0 or y >= n:
        return False
    if m[x,y] == -1:
        return False

    return True

def swap_tag(mm, i, j, _i, _j):
    t = mm[_i, _j]
    mm[_i,_j] = mm[i,j]
    mm[i,j] = t
    return mm


class Node(object):
    
    def __init__(self, m, steps=0):
        self.m = m
        self.steps = steps
        self.H = self.calc_value() + steps

        pos = np.where(m==start)
        self.x = pos[0][0]
        self.y = pos[1][0]


    def calc_value(self):
        # 计算所有结点的曼哈顿距离
        dist = 0
        for x in range(n):
            for y in range(n):
                if self.m[x,y] == -1:
                    continue
                ori_x, ori_y = self.m[x,y] // n, self.m[x,y] % n
                dist += abs(x-ori_x) + abs(y-ori_y)

        return dist


    def to_string(self):
        return "".join(["{:02d}".format(i) for i in self.m.reshape(-1)])


    def __lt__(self,other):
        return self.H < other.H

    def __gt__(self,other):
        return self.H > other.H


In [6]:
n = 4
m = np.arange(n*n).reshape(n,n)
# start = np.random.randint(n*n)
start = n*n-1
start_idx = (n-1, n-1)
print(start, start_idx)

step = 500

i, j = start_idx

dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]

while step > 0:
    _x, _y = dirs[np.random.randint(4)]
    if not can_access(i+_x, j+_y):
        continue
    
    step -= 1
    m = swap_tag(m, i, j, i+_x, j+_y)
    i, j = i+_x, j+_y

ori_m = m.copy()
m

15 (3, 3)


array([[ 7, 10,  8,  6],
       [ 3,  0, 13,  4],
       [15, 11,  1, 14],
       [ 9,  5, 12,  2]])

现在目标就是将 m 恢复到 ori_m，具体步骤就是 A-star 算法的范畴了。

In [7]:
max_search_step = 100000

open_list = [Node(ori_m.copy())]
clost_list = [] # 作为改进可以存放 hash

heapify(open_list)
while max_search_step > 0:
    max_search_step -= 1
    if len(open_list) > 0:
        cur = heappop(open_list)
    else:
        print("open list is empty now")
        break

    if cur.H == 0:
        print("find")
        break

    if cur.to_string() in clost_list:
        continue
    else:
        clost_list.append(cur.to_string())

    for i, j in dirs:
        if can_access(cur.x+i, cur.y+j):
            m_ = swap_tag(cur.m.copy(), cur.x, cur.y, cur.x+i, cur.y+j)
            heappush(open_list, Node(m_, cur.steps+1))


print("done", max_search_step)
    

IndexError: index 0 is out of bounds for axis 0 with size 0

另外一个问题就是对已经经过的路径进行判重，就是说之前出现过的状态不能再次出现，之前在处理 8 数码问题的时候，是使用的康托判重来计算的，但是既然想要拓展到 n 维的话，还是